# INDIAN PAINTINGS IMAGE CLASSIFICATION

## import library 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from PIL import UnidentifiedImageError

## Set Up Database Path

In [ ]:
dataset_path = 'paintings'

## Preprocess the Data

In [ ]:
img_width, img_height = 224, 224  # MobileNet input size
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


## Build the Transfer Learning Model with MobileNet

In [ ]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 8 classes


## Freeze the base model layers

In [ ]:
base_model.trainable = False

opt = Adam(lr=0.0001)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Train the Model

In [ ]:
# Step 5: Train the Model

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs= 25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    #callbacks=[reduce_lr, early_stop]
)
# Step 6: Evaluate the Model
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [8]:

model.save('MobileNet_model.h5')


C:\Program Files (x86)\Intel\openvino_env\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:


# Step 1: Load the Trained Model
model = load_model('MobileNet_model.h5')  # Replace with the path to your saved model

# Step 2: Load the Test Data
test_data_dir = 'C:\\Users\\Hemali Patel\\Desktop\\code unnati internship\\test imgae'  # Replace with the path to your test data folder
img_width, img_height = 224, 224  # Same as the input size for ResNet50V2
batch_size = 32

# Step 3: Get the Class Labels
class_labels = ['gond', 'kalighat', 'kangra', 'kerala_mural', 'madhubani', 'mandana', 'pichwai', 'warli']

# Step 4: Make Predictions on Test Data
def predict_image(image_path):
    try:
        img = image.load_img(image_path, target_size=(img_width, img_height))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match batch size
        img_array /= 255.0  # Rescale pixel values to 0-1

        prediction = model.predict(img_array)
        predicted_class = class_labels[np.argmax(prediction)]

        return predicted_class
    except UnidentifiedImageError as e:
        print(f"Error loading image: {image_path}, {e}")
        return None

test_images = os.listdir(test_data_dir)
for image_file in test_images:
    image_path = os.path.join(test_data_dir, image_file)
    if os.path.isfile(image_path):
        try:
            predicted_class = predict_image(image_path)
            if predicted_class is not None:
                print(f"Image: {image_file}, Predicted Class: {predicted_class}")
        except UnidentifiedImageError as e:
            print(f"Error processing image: {image_file}, {e}")


1/1 [==============================] - 1s 893ms/step
Image: gond painting_1.jpg, Predicted Class: gond
1/1 [==============================] - 0s 117ms/step
Image: gond painting_10.jpg, Predicted Class: gond
1/1 [==============================] - 0s 107ms/step
Image: gond painting_2.jpg, Predicted Class: gond
1/1 [==============================] - 0s 100ms/step
Image: gond painting_3.jpg, Predicted Class: gond
1/1 [==============================] - 0s 119ms/step
Image: gond painting_4.jpg, Predicted Class: gond
1/1 [==============================] - 0s 102ms/step
Image: gond painting_5.jpg, Predicted Class: gond
1/1 [==============================] - 0s 96ms/step
Image: gond painting_6.jpg, Predicted Class: gond
1/1 [==============================] - 0s 100ms/step
Image: gond painting_7.jpg, Predicted Class: gond
1/1 [==============================] - 0s 110ms/step
Image: gond painting_8.jpg, Predicted Class: gond
1/1 [==============================] - 0s 98ms/step
Image: gond painting_

1/1 [==============================] - 0s 116ms/step
Image: warli painting_6.jpg, Predicted Class: warli
1/1 [==============================] - 0s 127ms/step
Image: warli painting_7.jpg, Predicted Class: warli
1/1 [==============================] - 0s 108ms/step
Image: warli painting_8.jpg, Predicted Class: madhubani
1/1 [==============================] - 0s 109ms/step
Image: warli painting_9.jpg, Predicted Class: warli


In [10]:
import numpy as np
from sklearn.metrics import accuracy_score

# Step 7: Calculate Accuracy
def calculate_accuracy(model, validation_generator):
    y_true = []
    y_pred = []

    # Predict classes for validation data
    for i in range(len(validation_generator)):
        x_val, y_val = validation_generator[i]
        y_true.extend(np.argmax(y_val, axis=1))  # Actual labels
        y_pred.extend(np.argmax(model.predict(x_val), axis=1))  # Predicted labels

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

# Call the function to get accuracy
accuracy = calculate_accuracy(model, validation_generator)
print("Accuracy on validation data:", accuracy)


1/1 [==============================] - 1s 502ms/step
Accuracy on validation data: 0.7959183673469388


In [11]:
import numpy as np
from sklearn.metrics import accuracy_score

# Step 7: Calculate Accuracy
def calculate_accuracy(model, validation_generator):
    y_true = []
    y_pred = []

    # Predict classes for validation data
    for i in range(len(validation_generator)):
        x_val, y_val = validation_generator[i]
        y_true.extend(np.argmax(y_val, axis=1))  # Actual labels
        y_pred.extend(np.argmax(model.predict(x_val), axis=1))  # Predicted labels

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

# Call the function to get accuracy
accuracy = calculate_accuracy(model, validation_generator)
print("Accuracy on validation data:", accuracy)


1/1 [==============================] - 1s 501ms/step
Accuracy on validation data: 0.8095238095238095


In [12]:
import numpy as np
from sklearn.metrics import accuracy_score

# Step 7: Calculate Accuracy
def calculate_accuracy(model, validation_generator):
    y_true = []
    y_pred = []

    # Predict classes for validation data
    for i in range(len(validation_generator)):
        x_val, y_val = validation_generator[i]
        y_true.extend(np.argmax(y_val, axis=1))  # Actual labels
        y_pred.extend(np.argmax(model.predict(x_val), axis=1))  # Predicted labels

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

# Call the function to get accuracy
accuracy = calculate_accuracy(model, validation_generator)
print("Accuracy on validation data:", accuracy)


1/1 [==============================] - 1s 502ms/step
Accuracy on validation data: 0.8367346938775511


In [13]:
import numpy as np
from sklearn.metrics import accuracy_score

# Step 7: Calculate Accuracy
def calculate_accuracy(model, validation_generator):
    y_true = []
    y_pred = []

    # Predict classes for validation data
    for i in range(len(validation_generator)):
        x_val, y_val = validation_generator[i]
        y_true.extend(np.argmax(y_val, axis=1))  # Actual labels
        y_pred.extend(np.argmax(model.predict(x_val), axis=1))  # Predicted labels

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

# Call the function to get accuracy
accuracy = calculate_accuracy(model, validation_generator)
print("Accuracy on validation data:", accuracy)


1/1 [==============================] - 1s 511ms/step
Accuracy on validation data: 0.7891156462585034
